In [1]:
from tqdm import tqdm 
import torch
import random
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from utils.base import eval_accuracy
from utils.base import get_correct_predictions_subset
from utils.data import create_butterfly_dataset
from utils.data import create_imagenet_dataset
from model.imagenet_classifier import InceptionV3
from algo.attacker import adversarial_generator
from torch.utils.data import DataLoader
import torch.nn.functional as F
from utils.base import quick_predict
from algo.attacker import PIA_adversarial_generator
from algo.defender import PartialInfo
from tqdm import trange
from algo.defender import AAAProtectedClassifier

# (0) Download Tiny ImageNet 

In [ ]:
!kaggle datasets download -d akash2sharma/tiny-imagenet

In [ ]:
import zipfile
with zipfile.ZipFile('tiny-imagenet.zip', 'r') as zip_ref:
    zip_ref.extractall('data/tinyimagenet')

# (1) Process ImageNet Data

In [2]:
trainset, testset, normal_mapping, reverse_mapping, sample_img_dataset = create_imagenet_dataset(img_reshape=(3, 224, 224), num_classes = 20)
assert len(trainset) == 8000, 'Size of train set not match'
assert len(testset) == 2000, 'Size of test set not match'

# (2) Import Classifier

In [3]:
model = InceptionV3(num_classes=20).to('cuda')
model.load_state_dict(torch.load('./model/states/imagenetclassifier.pth'))

<All keys matched successfully>

# (3) Evaluate Untargeted Adversarial Examples

We also subset the parts where the model could provide correct predictions to attack. We subset 500 images for processing.

In [4]:
model.eval()
accuracy, correct_subset = get_correct_predictions_subset(model, testset, batch_size=100)
print('Accuracy on test set is {:.4f}'.format(accuracy))
print('Number of correctly predicted samples:', len(correct_subset))

Accuracy on test set is 0.9730
Number of correctly predicted samples: 1946


In [5]:
# subset only 500 for processing
subset_indices = correct_subset.indices
random_indices = random.sample(subset_indices, 500)
correct_subset = torch.utils.data.Subset(correct_subset.dataset, random_indices)

# (4) Parameter Definition

In [7]:
# attacker params
batch_size = 32
query_limit = 30000 # max attack limit 
LO_query_limit = 500000 # max attack limit 
search_var = 1e-3 # amount to perturb the input image
sample_num = 50 # 2*sample_num for estimating gradient
bound = 0.1 # the l-infinity distance between the adversarial example and the input image
# partial information paramters + lebel-only parameters
epsilon = 0.5 # initial searching range from the target image
delta = 0.01 # rate to decrease epsilon
eta_max = 0.02 # maximum learning rate
eta_min = 0.01 # minimum learning rate
k = 5 # information access
# label-only parameter
mu = 0.001 # radius for sampling ball
m = 50 # 2*number of sample for proxy score
#correct_subset_loader = DataLoader(correct_subset, batch_size = batch_size, shuffle = False)
correct_subset_loader = DataLoader(correct_subset, batch_size = 5, shuffle = False)
lr = 0.01

In [8]:
# defender params
alpha=1
tau=80
kappa=100
T=1
beta=5
lr=0.1

model.eval()
protected_model = AAAProtectedClassifier(model=model, alpha=alpha, tau=tau, kappa=kappa, T=T, beta=beta, lr=lr, AAA_type='linear')

C:\Users\jinga\anaconda3\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


# (5) Query-Limited Setting

In [13]:
from utils.base import quick_predict

protected_model.to('cuda')
protected_model.eval()

success_count = 0
query_counts = []
adv_images = []
with torch.no_grad():
    for i in trange(len(correct_subset)):
        images = correct_subset[i][0].unsqueeze(0)
        
        target_classes = torch.tensor(correct_subset[i][1]).view(1).to('cuda')
        adv_image_batch, query_count_batch = adversarial_generator(protected_model, target_classes, images, 
                                                                 search_var, sample_num,
                                                                bound, lr, query_limit)
        query_counts.append(query_count_batch)
        adv_images.append(adv_image_batch)
        
        adv_class = quick_predict(protected_model, adv_image_batch)
        success_count += (adv_class != target_classes).to(float).sum()
adv_image_all = torch.concat(adv_images, dim = 0)
query_count_all = torch.concat(query_counts, dim = 0)
torch.save(adv_image_all, "QL_adv_img_defense_net.pt")
torch.save(query_count_all, "QL_query_defense_net.pt")
print('Success count is {}, total count is {}, success attack rate is {}'.format(success_count, len(correct_subset), success_count /len(correct_subset) ))

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [35:40<00:00,  4.28s/it]


Success count is 485.0, total count is 500, success attack rate is 0.97


# (6) Partial-Information Setting

In [11]:
success_count_PIA = 0
query_counts_PIA = []
adv_images_PIA = []
with torch.no_grad():
    for i in trange(len(correct_subset) - 450):
        
        images = correct_subset[i][0].unsqueeze(0)
        # images = batch[0]
        adv_image_batch, query_count_batch = PIA_adversarial_generator(protected_model, images, sample_img_dataset,
                                                                      epsilon, delta, search_var,
                                                                      sample_num, eta_max, eta_min,
                                                                      bound, k, query_limit, label_only = False)
        query_counts_PIA.append(query_count_batch)
        adv_images_PIA.append(adv_image_batch)
        
        
        reach_bound = (query_count_batch < query_limit).to('cuda')
        successful_attack = (quick_predict(protected_model, adv_image_batch) != correct_subset[i][1]).to('cuda')
        success_count_PIA += (successful_attack * reach_bound).to(int).sum()
        
        
adv_image_all_pia = torch.concat(adv_images_PIA, dim = 0)
query_count_all_pia = torch.concat(query_counts_PIA, dim = 0)
torch.save(adv_image_all_pia, "PIA_adv_img_defense_net.pt")
torch.save(query_count_all_pia, "PIA_query.pt")
print('Success count is {}, total count is {}, success attack rate is {}'.format(success_count_PIA, len(correct_subset), success_count_PIA /len(correct_subset) ))

 22%|█████████████████▍                                                             | 11/50 [36:57<2:11:01, 201.58s/it]


KeyboardInterrupt: 

# (7) Label-only Setting

In [ ]:
success_count_LO = 0
query_counts_LO = []
adv_images_LO = []

with torch.no_grad():
    for i in trange(10):
        images = correct_subset[i][0].unsqueeze(0)
        adv_image_batch, query_count_batch = PIA_adversarial_generator(model, images, sample_img_dataset,
                                                                      epsilon, delta, search_var,
                                                                      sample_num, eta_max, eta_min,
                                                                      bound, k, query_limit=LO_query_limit, label_only = True, mu = mu, m = m) # different query_limit
        query_counts_LO.append(query_count_batch)
        adv_images_LO.append(adv_image_batch)
        
        reach_bound = (query_count_batch < LO_query_limit).to('cuda')
        successful_attack = (quick_predict(model, adv_image_batch) != correct_subset[i][1]).to('cuda')
        success_count_LO += (successful_attack * reach_bound).to(int).sum()

        
adv_image_all_lo = torch.concat(adv_images_LO, dim = 0)
query_count_all_lo = torch.concat(query_counts_LO, dim = 0)
torch.save(adv_image_all_lo, "LO_adv_img.pt")
torch.save(query_count_all_lo, "LO_query.pt")
print('Success count is {}, total count is {}, success attack rate is {}'.format(success_count_PIA, 10, success_count /10 ))